# Pasos seguidos para segmentar texto en tiempos de presionado

In [3]:
import string
import time
import numpy as np

## Paso 0: Definir las coordenadas de las teclas

In [17]:
us_layout ={
    "ctrl":[5,1],       "shift":[4,1],
    "capslock":[3,1],   "tab":[2,1],
    '`':[1,1],          '~':[1,1],
    'esc':[0,1],        'space':[5,7],
    'backspace':[1,15], 'return':[3,14],
    
    '1':[1,2],          '!':[1,2],
    '2':[1,3],          '@':[1,3],
    '3':[1,4],          '#':[1,4],
    '4':[1,5],          '$':[1,5],
    '5':[1,6],          '%':[1,6],
    '6':[1,7],          '^':[1,7],
    '7':[1,8],          '&':[1,8],
    '8':[1,9],          '*':[1,9],
    '9':[1,10],         '(':[1,10],
    '0':[1,11],         ')':[1,11],

    
    'q':[2,2],
    'w':[2,3],
    'e':[2,4],
    'r':[2,5],
    't':[2,6],
    'y':[2,7],
    'u':[2,8],
    'i':[2,9],
    'o':[2,10],
    'p':[2,11],
    'a':[3,3],
    's':[3,4],
    'd':[3,5],
    'f':[3,6],
    'g':[3,7],
    'h':[3,8],
    'j':[3,9],
    'k':[3,10],
    'l':[3,11],
    'z':[4,3],
    'x':[4,4],
    'c':[4,5],
    'v':[4,6],
    'b':[4,7],
    'n':[4,8],
    'm':[4,9],

    '-':[1,12],
    '_':[1,12],
    '+':[1,13],
    '=':[1,13],
    '[':[2,12],
    '{':[2,12],
    ']':[2,13],
    '}':[2,13],
    '\\':[2,14],
    '|':[2,14],
    ';':[3,11],
    ':':[3,11],
    "'":[3,12],
    '"':[3,12],
    ',':[4,10],
    '<':[4,10],
    '.':[4,11],
    '>':[4,11],
    '/':[4,12],
    '?':[4,12]
}
shifted_keys = '~!@#$%^&*()_+{}|:"<>?' + string.ascii_uppercase

## Paso 1: Preparar valores globales

In [8]:
key_to_index = {i:k for k,i in enumerate(us_layout.keys())}
index_to_key = {k:i for k,i in enumerate(us_layout.keys())}

# Usamos el mismo seed para generar resultados consistentes
person_seed = 1234
np.random.seed(person_seed)

# Handicap es un valor que representa la dificultad de la persona de teclear
handicap = np.random.uniform()

## Paso 2: Calcular tiempos normales de tecleo

In [27]:
# Calcular la matriz de tiempo entre pares de teclas

move_delay_matrix = np.empty((len(key_to_index), len(key_to_index)))
for k_i, i in key_to_index.items():
    for k_j, j in key_to_index.items():
        # We calculate the distance between the two keys
        pos_i = np.array(us_layout[k_i])
        pos_j = np.array(us_layout[k_j])
        distance = np.linalg.norm(pos_i - pos_j)

        move_delay_matrix[i][j] = distance

# Now we sum to the places where the destination is a letter, a scalar proportional to the handicap
for k, i in key_to_index.items():
    if k not in string.ascii_letters:
        continue

    move_delay_matrix[:][i] += np.random.uniform(low=handicap, high=handicap+0.3)

# Now we sum to the places where the destination is a number, a scalar proportional to the handicap
for k, i in key_to_index.items():
    if k not in string.digits:
        continue

    move_delay_matrix[:][i] += np.random.uniform(low=handicap+0.1, high=handicap+0.4)

# Now we sum to the places where the destination is a symbol, a scalar proportional to the handicap
for k, i in key_to_index.items():
    if k not in string.punctuation.split() + ["return", "backspace", "ctrl", "shift", "capslock", "tab", "esc", "space"]:
        continue

    move_delay_matrix[:][i] += np.random.uniform(low=handicap+0.2, high=handicap+0.6) 

move_delay_matrix

array([[ 0.40036392,  1.40036392,  2.40036392, ..., 10.45023954,
        11.44572494, 11.44572494],
       [ 1.63391479,  0.63391479,  1.63391479, ..., 10.63391479,
        11.63391479, 11.63391479],
       [ 2.77743806,  1.77743806,  0.77743806, ..., 10.82731368,
        11.82279908, 11.82279908],
       ...,
       [10.04987562, 10.        , 10.04987562, ...,  0.        ,
         1.        ,  1.        ],
       [11.04536102, 11.        , 11.04536102, ...,  1.        ,
         0.        ,  0.        ],
       [11.04536102, 11.        , 11.04536102, ...,  1.        ,
         0.        ,  0.        ]])

In [28]:
# Generar los tiempos que mantenemos presionados cada tecla
hold_delay_dict = {k: handicap * np.random.uniform(low=0.2,high=0.6) for k in us_layout.keys()}

hold_delay_dict

{'ctrl': 0.06611720283589612,
 'shift': 0.04110796015922441,
 'capslock': 0.1048108355575187,
 'tab': 0.0898477658685607,
 '`': 0.045004896255320816,
 '~': 0.10625238175071935,
 'esc': 0.09824676235720069,
 'space': 0.06261187665493052,
 'backspace': 0.10099309279491016,
 'return': 0.07718655715849673,
 '1': 0.03992746187831222,
 '!': 0.07151502962108537,
 '2': 0.07249494254505624,
 '@': 0.05659935932008467,
 '3': 0.10190717257242804,
 '#': 0.09535862038250655,
 '4': 0.08323274444483955,
 '$': 0.07606140216888486,
 '5': 0.07563914217422116,
 '%': 0.05873830266651483,
 '6': 0.08465244752412358,
 '^': 0.0960312017879885,
 '7': 0.05903274712329854,
 '&': 0.07831638516242756,
 '8': 0.04583662050100467,
 '*': 0.09297401157992836,
 '9': 0.1060282775472066,
 '(': 0.08174467051118169,
 '0': 0.11448891623671081,
 ')': 0.05199674710039231,
 'q': 0.03923851160445203,
 'w': 0.07331352920389086,
 'e': 0.10968333799318876,
 'r': 0.10311596339356287,
 't': 0.07456464116093262,
 'y': 0.107446627085926

## Paso 3: Creamos unas funciones auxiliares

In [29]:
def char_to_key(char):
    if char == ' ':
        intended_key = 'space'
    elif char == '\n':
        intended_key = 'return'
    elif char == '\t':
        intended_key = 'tab'
    else:
        intended_key = char.lower()

    return intended_key

In [30]:
def generate_typo(intended_key):
        intended_key = char_to_key(intended_key)

        # We generate a random displacement
        x_offset = np.random.randint(-1,2)
        y_offset = np.random.randint(-1,2)

        typo_coordinates = np.array(us_layout[intended_key]) - np.array((x_offset, y_offset))

        # Find posibilities
        typo_key = intended_key
        for k, v in us_layout.items():
            if np.all(np.array(v) == typo_coordinates) and k not in shifted_keys:
                typo_key = k

        return typo_key

## Paso 4: Dado un texto, generamos las teclas a teclear

In [31]:
text = "TEST test 123$%^ \t123\n"
stress = 0.5

In [35]:
keys = []
typo_posibility = stress * 0.2
for i, c in enumerate(text):
    last_key = char_to_key(text[i - 1])
    this_key = char_to_key(c)

    if i != 0 and np.random.uniform() <= typo_posibility and this_key not in ['tab', 'return']:
        # Add typo and keys
        max_time = int(5 * stress)
        time_to_notice = 0 if max_time <= 1 else np.random.randint(1, max_time)
        time_to_notice = time_to_notice if time_to_notice < len(text) - i else len(text) - i
        key = generate_typo(c)

        origin_i = key_to_index[last_key]
        dest_i = key_to_index[key]

        movement_delay = move_delay_matrix[origin_i][dest_i] * 0.1 * handicap

        hold_delay = hold_delay_dict[key]
        needs_shift = key in shifted_keys

        # A key action has: key, wheter it needs shift, movement shift and hold delay    
        keys.append( [key, needs_shift, movement_delay, hold_delay] )

        _c = char_to_key(text[i])
        for j in range(time_to_notice):
            _last_c = char_to_key(text[i + j - 1])
            _c = char_to_key(text[i + j])
            origin_i = key_to_index[_last_c]
            dest_i = key_to_index[_c]

            movement_delay = move_delay_matrix[origin_i][dest_i] * 0.1 * handicap

            hold_delay = hold_delay_dict[_c]
            needs_shift = _c in shifted_keys

            # A key action has: key, wheter it needs shift, movement shift and hold delay    
            keys.append( [_c, needs_shift, movement_delay, hold_delay] )

        # Backspace with delay
        origin_i = key_to_index[_c]
        dest_i = key_to_index['backspace']

        movement_delay = move_delay_matrix[origin_i][dest_i] * 0.1 * handicap
        hold_delay = hold_delay_dict['backspace']
        keys.append( ['backspace', False, movement_delay, hold_delay] )
        # Rest of backspaces
        for _ in range(time_to_notice):
            keys.append( ['backspace', False, 0.05 + np.random.uniform(-0.005, 0.005), hold_delay] )

    if i == 0:
        movement_delay = 0
    else:
        origin_i = key_to_index[last_key]
        dest_i = key_to_index[this_key]

        movement_delay = move_delay_matrix[origin_i][dest_i] * 0.1 * handicap

    hold_delay = hold_delay_dict[this_key]
    needs_shift = c in shifted_keys

    # A key action has: key, wheter it needs shift, movement shift and hold delay    
    keys.append( [this_key, needs_shift, movement_delay, hold_delay] )

In [36]:
keys

[['t', True, 0, 0.07456464116093262],
 ['e', True, 0.045958413993970194, 0.10968333799318876],
 ['s', True, 0.027718612398593453, 0.06019939746232982],
 ['t', True, 0.04683197928442698, 0.07456464116093262],
 ['space', False, 0.06821829186028149, 0.06261187665493052],
 ['t', False, 0.06947691813380875, 0.07456464116093262],
 ['e', False, 0.045958413993970194, 0.10968333799318876],
 ['s', False, 0.027718612398593453, 0.06019939746232982],
 ['t', False, 0.04683197928442698, 0.07456464116093262],
 ['space', False, 0.06821829186028149, 0.06261187665493052],
 ['1', False, 0.1315454336578093, 0.03992746187831222],
 ['e', False, 0.0539390206496524, 0.10968333799318876],
 ['2', False, 0.030265914681481515, 0.07249494254505624],
 ['backspace', False, 0.23839211378610067, 0.10099309279491016],
 ['backspace', False, 0.04617057869294775, 0.10099309279491016],
 ['2', False, 0.030265914681481515, 0.07249494254505624],
 ['3', False, 0.027720718369319143, 0.10190717257242804],
 ['$', True, 0.025928990